# Sistema de detección de enlaces spam
- El objetivo será implementar un sistema que sea capaz de detectar automáticamente si una página web contiene spam o no basándonos en su URL.

### Carga de datos
- Cargo la base de datos .csv utilizando pandas

In [24]:
import pandas as pd

total_data = pd.read_csv('https://breathecode.herokuapp.com/asset/internal-link?id=932&path=url_spam.csv')
total_data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


### Procesamiento de datos


In [25]:
# Convertimos la columna categórica/booleana a numérica (True=1, False=0)
total_data["is_spam"] = total_data["is_spam"].apply(lambda x: 1 if x == True else 0)

# Verificamos el balance de clases
print(total_data["is_spam"].value_counts())

is_spam
0    2303
1     696
Name: count, dtype: int64


Dado que una pagina web tiene otro tipo de caracteres (a diferencia del correo como se vio en la lectura de la academia), utilizaré otro tipo de preprocesado para separar palabras claras como "google, login, free etc."

In [27]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import os

nltk_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'nltk_data'))
nltk.download('stopwords')
nltk.download('wordnet')
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)


stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def preprocess_url(url):
    # 1. Eliminar prefijos de protocolo (https://, www.) para dejar solo el contenido
    url = re.sub(r'https?://(www\.)?', '', url)

    # 2. Dividir la URL por caracteres especiales (puntos, guiones, barras, etc.)
    tokens = re.split(r'[./\-_?=&]', url)

    # 3. Eliminar tokens vacíos o espacios
    tokens = [token for token in tokens if token]

    # 4. Lematización y eliminación de stopwords
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return " ".join(processed_tokens)

# Aplicamos la transformación
# Asumo que 'total_data' está definido en una celda anterior
total_data["url_cleaned"] = total_data["url"].apply(preprocess_url)

print(total_data[["url", "url_cleaned"]].head())

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060]
[nltk_data]     Se produjo un error durante el intento de conexión ya
[nltk_data]     que la parte conectada no respondió adecuadamente tras
[nltk_data]     un periodo de tiempo, o bien se produjo un error en la
[nltk_data]     conexión establecida ya que el host conectado no ha
[nltk_data]     podido responder>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] Se
[nltk_data]     produjo un error durante el intento de conexión ya que
[nltk_data]     la parte conectada no respondió adecuadamente tras un
[nltk_data]     periodo de tiempo, o bien se produjo un error en la
[nltk_data]     conexión establecida ya que el host conectado no ha
[nltk_data]     podido responder>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Usuario/nltk_data'
    - 'C:\\Users\\Usuario\\Documents\\Git Hub Repositories\\machine-learning-nlp\\.venv\\nltk_data'
    - 'C:\\Users\\Usuario\\Documents\\Git Hub Repositories\\machine-learning-nlp\\.venv\\share\\nltk_data'
    - 'C:\\Users\\Usuario\\Documents\\Git Hub Repositories\\machine-learning-nlp\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\Usuario\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Usuario\\Documents\\Git Hub Repositories\\machine-learning-nlp\\nltk_data'
******************************************************************

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# Vectorización (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000, max_df=0.8, min_df=5)
X = vectorizer.fit_transform(total_data["url_cleaned"]).toarray()
y = total_data["is_spam"]

# División en Train y Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Construyendo modelo SVM
- De momento entrenare un modelo con los parametros por defecto solamente utilizando la semilla randomstate 42

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Inicialización con parámetros por defecto
model = SVC(random_state=42)
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Evaluación inicial
print(f"Accuracy base: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

### Optimizacion del modelo (hiperparametros)
- Aquí usare GridSearchCV.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos la cuadrícula de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'] # Relevante para kernel rbf
}

grid_search = GridSearchCV(estimator=SVC(random_state=42), param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)

grid_search.fit(X_train, y_train)

print(f"Mejores parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor puntuación (accuracy): {grid_search.best_score_}")

# Guardamos el mejor modelo en una variable
best_model = grid_search.best_estimator_

### Guardado del modelo
- Utilizando la libreria pickle, guardare el modelo como svm_url_spam.pkl

In [ ]:
from pickle import dump

# Guardamos el modelo optimizado
dump(best_model, open("../models/svm_url_spam_optimized.sav", "wb"))
print("Modelo guardado exitosamente.")